<a href="https://colab.research.google.com/github/Ishakasar/Recommender-engine/blob/main/Isha_Kasar_Team_2_Capstone_Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Build a recommender engine that reviews customer ratings and purchase history to recommend items and improve sales. </u></b>

### Amazon.com is one of the largest electronic commerce and cloud computing companies.

### Just a few Amazon related facts:

### They lost $4.8 million in August 2013, when their website went down for 40 mins. They hold the patent on 1-Click buying, and licenses it to Apple. Their Phoenix fulfilment centre is a massive 1.2 million square feet. Amazon relies heavily on a Recommendation engine that reviews customer ratings and purchase history to recommend items and improve sales.


### This is a dataset related to over 2 Million customer reviews and ratings of Beauty related products sold on their website.

### It contains

* ### the unique UserId (Customer Identification),
* ### the product ASIN (Amazon's unique product identification code for each product),
* ### Ratings (ranging from 1-5 based on customer satisfaction) and
* ### the Timestamp of the rating (in UNIX time)

### This dataset contains product reviews and metadata from Amazon, including 142.8 million reviews spanning May 1996 - July 2014.

### This dataset includes reviews (ratings, text, helpfulness votes), product metadata (descriptions, category information, price, brand, and image features), and links (also viewed/also bought graphs).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD


In [4]:
df1=pd.read_csv('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/ratings_Beauty.csv')

In [5]:
df=pd.read_json('/content/drive/MyDrive/AlmaBetter/Cohort Aravali/Module 4/Week 4/reviews_Beauty_5.json.gz', lines = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198502 entries, 0 to 198501
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      198502 non-null  object
 1   asin            198502 non-null  object
 2   reviewerName    197116 non-null  object
 3   helpful         198502 non-null  object
 4   reviewText      198502 non-null  object
 5   overall         198502 non-null  int64 
 6   summary         198502 non-null  object
 7   unixReviewTime  198502 non-null  int64 
 8   reviewTime      198502 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.6+ MB


In [7]:
df.describe()

,overall,unixReviewTime
count,198502.000000,1.985020e+05
mean,4.190391,1.364450e+09
std,1.166580,3.841937e+07
min,1.000000,1.023840e+09
25%,4.000000,1.354493e+09
50%,5.000000,1.372205e+09
75%,5.000000,1.391213e+09
max,5.000000,1.406074e+09


In [8]:
df.isnull().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [9]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198502 entries, 0 to 198501
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      198502 non-null  object
 1   asin            198502 non-null  object
 2   reviewerName    197116 non-null  object
 3   helpful         198502 non-null  object
 4   reviewText      198502 non-null  object
 5   overall         198502 non-null  int64 
 6   summary         198502 non-null  object
 7   unixReviewTime  198502 non-null  int64 
 8   reviewTime      198502 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.6+ MB


In [11]:
df.describe()

,overall,unixReviewTime
count,198502.000000,1.985020e+05
mean,4.190391,1.364450e+09
std,1.166580,3.841937e+07
min,1.000000,1.023840e+09
25%,4.000000,1.354493e+09
50%,5.000000,1.372205e+09
75%,5.000000,1.391213e+09
max,5.000000,1.406074e+09


In [12]:
df.isnull().sum()

reviewerID           0
asin                 0
reviewerName      1386
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [13]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [14]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014"
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014"
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013"
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013"
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013"


In [15]:
df.rename(columns={"reviewerID": "UserId", "asin": "ProductId"}, inplace=True)
df['Rating'] = df1['Rating']


In [16]:
df.shape

(198502, 10)

In [17]:
df.dropna(inplace = True)

In [18]:
df.shape

(197116, 10)

In [19]:
df.head()

,UserId,ProductId,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Rating
0,A1YJEY40YUW4SE,7806397051,Andrea,"[3, 4]",Very oily and creamy. Not at all what I expect...,1,Don't waste your money,1391040000,"01 30, 2014",5.0
1,A60XNB876KYML,7806397051,Jessica H.,"[1, 1]",This palette was a decent price and I was look...,3,OK Palette!,1397779200,"04 18, 2014",3.0
2,A3G6XNM240RMWA,7806397051,Karen,"[0, 1]",The texture of this concealer pallet is fantas...,4,great quality,1378425600,"09 6, 2013",5.0
3,A1PQFP6SAJ6D80,7806397051,Norah,"[2, 2]",I really can't tell what exactly this thing is...,2,Do not work on my face,1386460800,"12 8, 2013",4.0
4,A38FVHZTNQ271F,7806397051,Nova Amor,"[0, 0]","It was a little smaller than I expected, but t...",3,It's okay.,1382140800,"10 19, 2013",1.0


In [20]:
df.drop(['reviewerName',	'helpful',	'reviewText',	'overall',	'summary',	'unixReviewTime',	'reviewTime'], axis = 1, inplace = True)

In [22]:
users_interactions_count_df = df.groupby(['UserId', 'ProductId']).size().groupby('UserId').size()
print('# of users: %d' % len(users_interactions_count_df))

interaction_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['UserId']]
print('# of users with at least 5 interactions: %d' % len(interaction_df))

# of users: 22345
# of users with at least 5 interactions: 22110


In [23]:
users_interactions_count_df.head()

UserId
A00414041RD0BXM6WK0GX    6
A00473363TJ8YSZ3YAGG9    6
A00700212KB3K0MVESPIY    9
A0078719IR14X3NNUG0F     8
A01198201H0E3GHV2Z17I    7
dtype: int64

In [24]:
interaction_df.head()

,UserId
0,A00414041RD0BXM6WK0GX
1,A00473363TJ8YSZ3YAGG9
2,A00700212KB3K0MVESPIY
3,A0078719IR14X3NNUG0F
4,A01198201H0E3GHV2Z17I


In [25]:
interactions_from_selected_users_df = df.merge(interaction_df, 
               how = 'right',
               left_on = 'UserId',
               right_on = 'UserId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions from users with at least 5 interactions: 196257


In [26]:
interactions_from_selected_users_df.head(20)

,UserId,ProductId,Rating
0,A00414041RD0BXM6WK0GX,B007IY97U0,5.0
1,A00414041RD0BXM6WK0GX,B00870XLDS,1.0
2,A00414041RD0BXM6WK0GX,B008MIRO88,5.0
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,5.0
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,5.0
5,A00414041RD0BXM6WK0GX,B00HFP4JZU,5.0
6,A00473363TJ8YSZ3YAGG9,B000052YQU,5.0
7,A00473363TJ8YSZ3YAGG9,B0019LVFI0,5.0
8,A00473363TJ8YSZ3YAGG9,B001L2BEWE,5.0
9,A00473363TJ8YSZ3YAGG9,B0020HEBX8,5.0


In [27]:
df.shape

(197116, 3)

In [28]:
interactions_from_selected_users_df.shape

(196257, 3)

In [29]:
import math
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df.groupby(['UserId', 'ProductId'])['Rating'].sum().apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head()

# of unique user/item interactions: 196257


,UserId,ProductId,Rating
0,A00414041RD0BXM6WK0GX,B007IY97U0,2.584963
1,A00414041RD0BXM6WK0GX,B00870XLDS,1.000000
2,A00414041RD0BXM6WK0GX,B008MIRO88,2.584963
3,A00414041RD0BXM6WK0GX,B00BQYYMN0,2.584963
4,A00414041RD0BXM6WK0GX,B00GRTQBTM,2.584963


In [30]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['UserId'], 
                                   test_size=0.30,
                                   random_state=42)

In [31]:

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 137379
# interactions on Test set: 58878


##Creating pivot table

In [32]:
#Creating a sparse pivot table with users in rows and items in columns
pivot_matrix_df = interactions_train_df.pivot(index='UserId', 
                                                          columns='ProductId', 
                                                          values='Rating').fillna(0)

In [33]:
pivot_matrix_df.shape

(22110, 12090)

In [34]:
df.shape

(197116, 3)

In [35]:
pivot_matrix_df.head()

ProductId,7806397051,9759091062,9788072216,9790790961,9790794231,B00004TMFE,B00004TUBL,B00004TUBV,B00004U9UY,B00004U9V2,B000050B6U,B000052WY7,B000052WYD,B000052WYL,B000052WYN,B000052X9G,B000052XIA,B000052XW5,B000052XYQ,B000052XZP,B000052XZX,B000052Y25,B000052Y33,B000052Y5F,B000052Y6Z,B000052YJC,B000052YJD,B000052YJH,B000052YJM,B000052YJX,B000052YKI,B000052YKM,B000052YKS,B000052YKY,B000052YL1,B000052YLX,B000052YM0,B000052YM3,B000052YM4,B000052YM7,...,B00KFM16OW,B00KGDRCZW,B00KH6F6TM,B00KHGIK54,B00KHH2VOY,B00KI5UL8I,B00KIEGBHE,B00KK1V28I,B00KK62714,B00KKKW03U,B00KLA4INE,B00KLJDYL2,B00KLZO2JE,B00KNIL36E,B00KNJOLSA,B00KPKIV8I,B00KQ4PEBU,B00KQBR9FM,B00KQVTX06,B00KTAJAIY,B00KTLBDYM,B00KTLBEEQ,B00KTP8Q1G,B00KVE7R0Q,B00KVO2JWC,B00KWE08Q0,B00KXG8JF4,B00L0C529Q,B00L2K53BS,B00L2KVF9W,B00L3K91OW,B00L3LB0IG,B00L3LBUGI,B00L5BXG36,B00L5JHZJO,B00L5KTZ0K,B00L6Q3BH6,B00LCEROA2,B00LG63DOM,B00LLPT4HI
UserId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A00414041RD0BXM6WK0GX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00473363TJ8YSZ3YAGG9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00700212KB3K0MVESPIY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0078719IR14X3NNUG0F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A01198201H0E3GHV2Z17I,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
pivot_matrix_df.shape

(22110, 12090)

In [37]:
pivot_matrix = pivot_matrix_df.values
pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
users_ids = list(pivot_matrix_df.index)
users_ids[:10]

['A00414041RD0BXM6WK0GX',
 'A00473363TJ8YSZ3YAGG9',
 'A00700212KB3K0MVESPIY',
 'A0078719IR14X3NNUG0F',
 'A01198201H0E3GHV2Z17I',
 'A02155413BVL8D0G7X6DN',
 'A02849582PREZYNEI31CV',
 'A029527620Q3SK5XW16RR',
 'A03236882LUP4ARMDDMXU',
 'A03364251DGXSGA9PSR99']

In [39]:
from scipy.sparse.linalg import svds
# The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 30

#Performs matrix factorization of the original user item matrix
U, sigma, VT = svds(pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [40]:
U.shape

(22110, 30)

In [41]:
sigma = np.diag(sigma)
sigma.shape

(30, 30)

In [42]:
VT.shape

(30, 12090)

In [43]:
'''SVD = TruncatedSVD(n_components=30)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape'''

'SVD = TruncatedSVD(n_components=30)\ndecomposed_matrix = SVD.fit_transform(X)\ndecomposed_matrix.shape'

In [44]:
all_usera_predicted_ratings = np.dot(np.dot(U, sigma), VT) 
all_usera_predicted_ratings

array([[ 2.93680940e-05,  1.13056164e-05,  1.81921854e-05, ...,
        -5.22967975e-07, -8.71978300e-06,  7.25330283e-05],
       [ 1.98348394e-05, -4.82075315e-06,  1.55185125e-05, ...,
        -3.51839030e-06, -5.86704743e-05, -9.81381477e-05],
       [ 3.98389217e-05,  1.30943631e-04,  1.39777348e-04, ...,
         2.38251182e-06, -6.21409525e-05,  7.92483265e-04],
       ...,
       [ 7.90191417e-04,  1.07001617e-04,  6.38769606e-04, ...,
        -2.30200752e-05,  2.37571687e-04,  1.49254389e-03],
       [-5.46953630e-06,  9.42965937e-05,  2.12922560e-04, ...,
         3.38088628e-06, -8.82271816e-05,  8.14853409e-04],
       [ 1.21347634e-04,  1.07752608e-04,  2.91900650e-04, ...,
         1.72136534e-05,  1.46904995e-04,  2.54603918e-03]])

In [49]:
#Converting the reconstructed matrix back to a Pandas dataframe
preds_df = pd.DataFrame(all_usera_predicted_ratings, columns = pivot_matrix_df.columns, index=users_ids).T
preds_df.head()

,A00414041RD0BXM6WK0GX,A00473363TJ8YSZ3YAGG9,A00700212KB3K0MVESPIY,A0078719IR14X3NNUG0F,A01198201H0E3GHV2Z17I,A02155413BVL8D0G7X6DN,A02849582PREZYNEI31CV,A029527620Q3SK5XW16RR,A03236882LUP4ARMDDMXU,A03364251DGXSGA9PSR99,A0388397363MZHRU6ALSX,A042274212BJJVOBS4Q85,A04371222UWPCRS81JO2Z,A0459829SI5B5WIOZZHX,A0508779FEO1DUNOSQNX,A05306962T0DL4FS2RA7L,A059547920Q3LZVFHLPI3,A0614215ICIJ7IVV472R,A06298653TJ91G7R7MD6R,A0647567NOSE5H95OBQ2,A06586901ZOUBJS3R268N,A0733801U845JJGV4H8V,A07420391OVT9BFTVMUOI,A08409921GVN7C8RD97CV,A0864963DOAY7LXGS5I6,A090214335ZYBWI945UJI,A0908131Z7BWYSMRQ16T,A09149312G3TQJJF59GC8,A09386383518NVR7RSA4F,A099766128UI0NCS98N1E,A100E9O4Z9BP4Q,A100L918633LUO,A100LIFNJYZWW7,A100UD67AHFODS,A100VQNP6I54HS,A100WO06OQR8BQ,A100X30Z2GCTEM,A100ZQDV7L8PVV,A1010QRG4BH51B,A101Q13OPIP7ZR,...,AZRL9I4SG5ADA,AZRPMQ8A4H0W4,AZS23E3IKSF9D,AZS4UNYSAZVG8,AZS5TOKUYLDO0,AZS9WNWY48XF5,AZSWYOYUCD81E,AZTYMMD36CXY2,AZTZ7SIIRXLXE,AZU2UCJP6WUQO,AZUEKJ6P9TFSJ,AZUH2MX87LX7J,AZUH5XSJ2NLXR,AZUPWQFWXZYE9,AZV2AG96CRJ26,AZV7CNMMJERKW,AZVM27SJPO3GL,AZW9UD7Z7J0WL,AZWJAXOQMB8EK,AZWXG6KBXXC2N,AZX1JTTIUYZX4,AZX5U35YDZZXI,AZXH0JC9KOLU8,AZXP46IB63PU8,AZY3Z9QI0G8L,AZY8RW6L7W52Z,AZYA6NBTF2843,AZYAFW3CRARM6,AZYH32578YZBF,AZYN9RRLZUNVA,AZYNK58DULI59,AZYVVBYSD74KP,AZZ5ASC403N74,AZZ91VNEU3XYX,AZZIOU48NE8PC,AZZNK89PXD006,AZZQXL8VDCFTV,AZZT1ERHBSNQ8,AZZU6NXB8YJN9,AZZZLM1E5JJ8C
ProductId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7806397051,0.000029,0.000020,0.000040,0.000585,-0.001250,0.000054,0.000469,0.000392,-0.000014,0.001318,0.000051,0.000306,0.000057,0.000007,0.000176,0.000441,0.000989,0.000004,0.000084,1.038767e-04,0.000069,0.000178,2.094256e-05,4.296125e-06,0.000303,0.000205,-0.000241,0.000212,0.000212,0.000654,0.000657,0.000016,0.000082,3.882717e-06,0.000332,-0.000082,0.000022,-0.000185,-0.000422,0.000023,...,-0.000084,-0.000968,0.000036,0.000173,0.000409,0.000579,0.000028,0.000011,0.000013,7.454558e-05,0.000007,-0.000335,6.021255e-05,0.000076,0.000170,-0.000808,-0.000008,0.000048,0.000078,0.000034,0.000022,-1.071828e-05,6.686231e-06,0.000516,0.000030,0.000242,0.000194,0.000057,0.000023,0.000181,4.439825e-06,0.000107,0.000052,0.000341,0.000015,0.000026,0.000192,0.000790,-0.000005,0.000121
9759091062,0.000011,-0.000005,0.000131,0.000244,-0.000758,-0.000314,0.000149,-0.000552,-0.000207,0.000396,-0.000005,0.000248,-0.000005,0.000054,0.000252,0.000605,0.000215,0.000003,0.000016,-7.748004e-05,0.000140,-0.000017,-4.333169e-07,2.704785e-06,0.000014,0.000054,0.000460,0.000016,0.000099,0.000340,0.000306,-0.000006,0.000088,-1.341853e-06,0.000911,0.013498,-0.000039,-0.001016,0.000218,-0.000002,...,0.000528,-0.000501,-0.000007,0.000132,0.000588,0.000280,0.000012,0.000003,0.000192,-2.281121e-05,-0.000001,0.000120,-4.338206e-07,-0.000082,0.000135,0.000099,-0.000003,0.000063,0.000067,0.000074,-0.000009,3.089607e-07,-3.280944e-06,0.000020,-0.000032,0.000155,-0.000024,-0.000050,-0.000014,0.000035,-2.789017e-05,-0.000024,-0.000033,0.000677,0.000002,0.000009,-0.000015,0.000107,0.000094,0.000108
9788072216,0.000018,0.000016,0.000140,0.000002,0.000168,0.000093,0.000004,0.000324,0.000455,0.000938,0.000041,0.000543,-0.000002,0.000024,0.000415,0.001012,0.000723,0.000009,0.000077,7.028314e-04,0.000406,0.000044,3.206382e-05,9.539685e-06,0.000368,0.000180,0.000057,0.000189,0.000131,0.001027,0.000387,0.000032,0.000111,5.670159e-06,0.001183,-0.001596,0.000100,0.000092,-0.000002,0.000070,...,-0.000050,0.000193,0.000194,0.000125,0.001024,0.000188,0.000078,0.000041,0.000515,3.139124e-05,0.000025,0.000038,2.673948e-04,-0.000143,0.000136,0.000089,0.000018,0.000092,0.000179,0.000080,0.000080,1.365184e-05,3.779088e-05,0.001274,0.000081,0.000329,0.000211,0.000114,0.000031,0.000162,2.292486e-05,0.000119,0.000671,0.000960,-0.000004,0.000087,0.000363,0.000639,0.000213,0.000292
9790790961,0.000015,0.000006,0.000195,-0.000235,0.000360,0.000059,-0.000194,-0.000033,0.000057,0.003083,0.000040,0.000275,-0.000018,0.000044,0.000382,0.000262,0.000361,0.000006,0.

In [50]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df):
        self.cf_predictions_df = cf_predictions_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False).reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating content that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['ProductId'].isin(items_to_ignore)].sort_values('recStrength', ascending = False).head(topn)

        return recommendations_df


cf_recommender_model = CFRecommender(preds_df)     

In [52]:
#Indexing by personId to speed up the searches during evaluation
purchases_full_indexed_df = interactions_full_df.set_index('UserId')
purchases_train_indexed_df = interactions_train_df.set_index('UserId')
purchases_test_indexed_df = interactions_test_df.set_index('UserId')

In [53]:
import random

In [57]:
def get_product_purchased(UserId, df):
    purchased_product = df.loc[UserId]['ProductId']
    return set(purchased_product if type(purchased_product) == pd.Series else [purchased_product])

In [58]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:

    # Function for getting the set of items which a user has not interacted with
    def get_not_purchased_products_sample(self, UserId, sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, seed=42):
        purchased_products = get_product_purchased(UserId, purchases_full_indexed_df)
        all_products = set(df['ProductId'])
        non_purchased_products = all_products - purchased_products

        random.seed(seed)
        non_purchased_products_sample = random.sample(non_purchased_products, sample_size)
        return set(non_purchased_products_sample)

    # Function to verify whether a particular item_id was present in the set of top N recommended items
    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index
    
    # Function to evaluate the performance of model for each user
    def evaluate_model_for_user(self, model, person_id):
        
        # Getting the items in test set
        purchased_values_testset = purchases_test_indexed_df.loc[person_id]
        
        if type(purchased_values_testset['ProductId']) == pd.Series:
            users_purchased_products_testset = set(purchased_values_testset['ProductId'])
        else:
            users_purchased_products_testset = set([(purchased_values_testset['ProductId'])])
            
        purchased_products_count_testset = len(users_purchased_products_testset) 

        # Getting a ranked recommendation list from the model for a given user
        person_recs_df = model.recommend_items(person_id, items_to_ignore=get_product_purchased(person_id, purchases_train_indexed_df),topn=10000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        hits_at_15_count = 0
        
        # For each item the user has interacted in test set
        for item_id in users_purchased_products_testset:
            
            # Getting a random sample of 100 items the user has not interacted with
            non_purchased_products_sample = self.get_not_purchased_products_sample(person_id,seed=item_id)

            # Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_purchased_products_sample.union(set([item_id]))

            # Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['ProductId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['ProductId'].values
            
            # Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
            hit_at_15, index_at_15 = self._verify_hit_top_n(item_id, valid_recs, 15)
            hits_at_15_count += hit_at_15

        # Recall is the rate of the interacted items that are ranked among the Top-N recommended items
        recall_at_5 = hits_at_5_count / float(purchased_products_count_testset)
        recall_at_10 = hits_at_10_count / float(purchased_products_count_testset)
        recall_at_15 = hits_at_15_count / float(purchased_products_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'hits@15_count':hits_at_15_count,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10,
                          'recall@15': recall_at_15,
                          'interacted_count': purchased_products_count_testset}
        return person_metrics

    
    # Function to evaluate the performance of model at overall level
    def evaluate_model(self, model):
        
        people_metrics = []
        
        for idx, person_id in enumerate(list(purchases_test_indexed_df.index.unique().values)):    
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['UserId'] = person_id
            people_metrics.append(person_metrics)
            
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics).sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_15 = detailed_results_df['hits@15_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10,
                          'recall@15': global_recall_at_15}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()    

In [59]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
22109 users processed


In [60]:
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)


Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.365773293929821, 'recall@10': 0.45799789395020213, 'recall@15': 0.5215360576106526}


,hits@5_count,hits@10_count,hits@15_count,recall@5,recall@10,recall@15,interacted_count,UserId
3136,13,18,25,0.213115,0.295082,0.409836,61,A2V5R832QCSOMX
1132,53,53,54,0.929825,0.929825,0.947368,57,ALNFHVS3SC4FV
289,20,34,40,0.370370,0.629630,0.740741,54,AKMEY1BSHSDG7
3635,6,11,14,0.130435,0.239130,0.304348,46,A3KEZLJ59C1JVH
1643,5,11,11,0.111111,0.244444,0.244444,45,ALQGOMOY1F5X9
1548,40,41,42,0.888889,0.911111,0.933333,45,A1UQBFCERIP7VJ
622,27,31,31,0.600000,0.688889,0.688889,45,A34BZM6S9L7QI4
2070,32,35,35,0.820513,0.897436,0.897436,39,A3NHUQ33CFH3VM
1508,34,35,35,0.918919,0.945946,0.945946,37,AEL6CQNQXONBX
1336,6,8,13,0.166667,0.222222,0.361111,36,A281NPSIMI1C2R
